Pip install preperation

In [1]:
!pip install datasets>=2.6.1 git+https://github.com/huggingface/transformers librosa evaluate>=0.30 jiwer accelerate>=0.20.1


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-chax89rd


Mount Gdrive

In [2]:
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import os
import shutil
# According to this issue it should be fine to clear the cache while it's unmounted.
# https://github.com/googlecolab/colabtools/issues/960
def clear_drivefs_cache():
    base_path = '/root/.config/Google/DriveFS'

    # Check if the base path exists
    if not os.path.exists(base_path):
        print("DriveFS path does not exist.")
        return

    # Iterate through subdirectories to find the content_cache folder
    for subdir in os.listdir(base_path):
        cache_path = os.path.join(base_path, subdir, 'content_cache')
        if os.path.exists(cache_path):
            print(f"Clearing cache in: {cache_path}")
            # Remove the content_cache directory
            shutil.rmtree(cache_path)
            # Recreate the content_cache directory after clearing
            os.makedirs(cache_path)
            print("Cache cleared.")
            return

    print("Content cache directory not found.")


In [4]:
def remountGDrive():
  drive.flush_and_unmount()
  # !find / -type f -exec du -h {} + | sort -rh | head -n 100
  clear_drivefs_cache()
  drive.mount('/content/drive')

# Disbale caching since that lead to some errors

In [5]:
from datasets import disable_caching
disable_caching()


## Load Dataset

In [6]:
import os

dir_path = "/content/drive/MyDrive/Lab2/ColabData/Second"

# agregate all of the subdir's

subdir_list = [os.path.join(dir_path, d) for d in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, d))]
subdir_list_training = [subdir for subdir in subdir_list if "test" in subdir]
subdir_list = [subdir for subdir in subdir_list if "test" not in subdir]


print(len(subdir_list))
print(len(subdir_list_training))
print(subdir_list)
print(subdir_list_training)
# These commands take a while so not worth everytime
# Output with filled drive should be:
# 443G	/content/drive/MyDrive/Lab2/ColabData/Second
#print("Total storage used by directory: ")
#!du -hs "/content/drive/MyDrive/Lab2/ColabData/Second"


497
17
['/content/drive/MyDrive/Lab2/ColabData/Second/processed_chunk_1.dataset', '/content/drive/MyDrive/Lab2/ColabData/Second/processed_chunk_2.dataset', '/content/drive/MyDrive/Lab2/ColabData/Second/processed_chunk_3.dataset', '/content/drive/MyDrive/Lab2/ColabData/Second/processed_chunk_4.dataset', '/content/drive/MyDrive/Lab2/ColabData/Second/processed_chunk_5.dataset', '/content/drive/MyDrive/Lab2/ColabData/Second/processed_chunk_6.dataset', '/content/drive/MyDrive/Lab2/ColabData/Second/processed_chunk_7.dataset', '/content/drive/MyDrive/Lab2/ColabData/Second/processed_chunk_8.dataset', '/content/drive/MyDrive/Lab2/ColabData/Second/processed_chunk_9.dataset', '/content/drive/MyDrive/Lab2/ColabData/Second/processed_chunk_10.dataset', '/content/drive/MyDrive/Lab2/ColabData/Second/processed_chunk_11.dataset', '/content/drive/MyDrive/Lab2/ColabData/Second/processed_chunk_12.dataset', '/content/drive/MyDrive/Lab2/ColabData/Second/processed_chunk_13.dataset', '/content/drive/MyDrive/La

Now that we have the list of datasets as files we can start reading them.
For this we will create a few helper functions.

The keep_in_memory flag of the load from disk function will force the datasets to be loaded into memory and not just be lacily evaluated.
We also clean up the intermediate variables since the individual datasets are very large in size.


In [28]:
from datasets import load_from_disk, concatenate_datasets
from datasets import load_dataset, DatasetDict
def read_folder(path):
    dataset = load_from_disk(path)
    return dataset



# To save snapshots of the trained model.
def save_snapshot(trainer, save_path, step):
    snapshot_path = os.path.join(save_path, f"snapshot_step_{step}")
    trainer.save_model(snapshot_path)
    print("saved snapshot to: ", save_path)

import gc

def read_datasets_in_chunks(sub_dirs, chunk_size, offset=0):
    """
    Generator function that yields datasets in chunks from a list of subdirectories,
    with the option to skip a number of chunks at the beginning.

    Parameters:
    - sub_dirs: A list of subdirectories containing the datasets.
    - chunk_size: The number of datasets to read at a time.
    - offset: The number of chunks to skip before starting to yield datasets.

    Yields:
    - A tuple of the current chunk index and a list of datasets, where the length of the list is up to chunk_size.
    """
    start_index = offset * chunk_size
    for i in range(start_index, len(sub_dirs), chunk_size):
        chunk_dirs = sub_dirs[i:i + chunk_size]
        print(f"Loading datasets from {chunk_size} directories: {chunk_dirs}")

        # Assuming read_folder reads a single dataset and concatenate_datasets
        # is a function that concatenates a list of datasets:
        datasets = [read_folder(dir) for dir in chunk_dirs]
        concatenated_datasets = concatenate_datasets(datasets)

        yield i // chunk_size, concatenated_datasets  # Yielding the chunk index

        # Clear the list of datasets to help free memory.
        del datasets
        del concatenated_datasets

        # Suggest to the garbage collector to free up memory.
        gc.collect()


Prerequisite for training

In [8]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="German", task="transcribe")

from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="German", task="transcribe")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

## Training and Evaluation

Now that we've prepared our data, we're ready to dive into the training pipeline.
The [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer)
will do much of the heavy lifting for us. All we have to do is:

- Define a data collator: the data collator takes our pre-processed data and prepares PyTorch tensors ready for the model.

- Evaluation metrics: during evaluation, we want to evaluate the model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric. We need to define a `compute_metrics` function that handles this computation.

- Load a pre-trained checkpoint: we need to load a pre-trained checkpoint and configure it correctly for training.

- Define the training configuration: this will be used by the 🤗 Trainer to define the training schedule.

Once we've fine-tuned the model, we will evaluate it on the test data to verify that we have correctly trained it
to transcribe speech in Hindi.

### Define a Data Collator

The data collator for a sequence-to-sequence speech model is unique in the sense that it
treats the `input_features` and `labels` independently: the  `input_features` must be
handled by the feature extractor and the `labels` by the tokenizer.

The `input_features` are already padded to 30s and converted to a log-Mel spectrogram
of fixed dimension by action of the feature extractor, so all we have to do is convert the `input_features`
to batched PyTorch tensors. We do this using the feature extractor's `.pad` method with `return_tensors=pt`.

The `labels` on the other hand are un-padded. We first pad the sequences
to the maximum length in the batch using the tokenizer's `.pad` method. The padding tokens
are then replaced by `-100` so that these tokens are **not** taken into account when
computing the loss. We then cut the BOS token from the start of the label sequence as we
append it later during training.

We can leverage the `WhisperProcessor` we defined earlier to perform both the
feature extractor and the tokenizer operations:

In [9]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [10]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metrics

We'll use the word error rate (WER) metric, the 'de-facto' metric for assessing
ASR systems. For more information, refer to the WER [docs](https://huggingface.co/metrics/wer). We'll load the WER metric from 🤗 Evaluate:

In [11]:
import evaluate

metric = evaluate.load("wer")

We then simply have to define a function that takes our model
predictions and returns the WER metric. This function, called
`compute_metrics`, first replaces `-100` with the `pad_token_id`
in the `label_ids` (undoing the step we applied in the
data collator to ignore padded tokens correctly in the loss).
It then decodes the predicted and label ids to strings. Finally,
it computes the WER between the predictions and reference labels:

In [12]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Load a Pre-Trained Checkpoint

Now let's load the pre-trained Whisper `small` checkpoint. Again, this
is trivial through use of 🤗 Transformers!

In [13]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.84k [00:00<?, ?B/s]



Override generation arguments - no tokens are forced as decoder outputs (see [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)), no tokens are suppressed during generation (see [`suppress_tokens`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.suppress_tokens)):

In [14]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

### Define the Training Configuration

For the hindi dataset: 13.9k rows
Training total: rows x 4000 iterations

For the german dataset: 496k rows
Chunked: 496x1000rows
training total= 496chunks * N iterations

The Hindi dataset is approximately 1/35th the size of the German dataset (13.9k vs. 496k).
If we naively scale the iterations by the same factor, we might suggest
4000 × 35 = 140,000 iterations for the entire German dataset.
If we allocate 140,000 iterations over 496 chunks, that's roughly 140,000/496≈282
iterations per chunk.

We decided on 110 steps: William will explain


In the final step, we define all the parameters related to training. For more detail on the training arguments, refer to the Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [15]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    # We keep the output_dir local because that lead to issues in development
    output_dir="whisper-small-de-max-william",  # change to a repo name of your choice
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,  # increase by 2x for every 2x decrease in batch size
    learning_rate=5e-5,
    warmup_steps=10,
    max_steps=110,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=110,
    save_steps=110, # 1000
    eval_steps=110, # We only evaluate once per run.
    logging_steps=10,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

**Note**: if one does not want to upload the model checkpoints to the Hub,
set `push_to_hub=False`.

We can forward the training arguments to the 🤗 Trainer along with our model,
dataset, data collator and `compute_metrics` function:

In [23]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=[],
    eval_dataset=[],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

*We*'ll save the processor object once before starting training. Since the processor is not trainable, it won't change over the course of training:

In [24]:
processor.save_pretrained(training_args.output_dir)

### Training

Training will take approximately 5-10 hours depending on your GPU or the one
allocated to this Google Colab. If using this Google Colab directly to
fine-tune a Whisper model, you should make sure that training isn't
interrupted due to inactivity. A simple workaround to prevent this is
to paste the following code into the console of this tab (_right mouse click_
-> _inspect_ -> _Console tab_ -> _insert code_).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton, 60000);
```

# Load the test data

For each iteration we get a random testdataset and validate with them

In [25]:
from datasets import load_from_disk, concatenate_datasets
import copy

# We could maybe think about just loading the test from the original source.
# That way we can avoid loading the whole dataset and then just extracting the test data


datasets = []
for dir in subdir_list_training:
    dataset = load_from_disk(dir)
    datasets.append(dataset)

dataset_test = concatenate_datasets(datasets)

#data_set_test =  copy.deepcopy(dataset_test['test'])
#del dataset_test
print(len(dataset_test))
dataset_test




16082


Dataset({
    features: ['input_features', 'labels'],
    num_rows: 16082
})

For each of the runs I will get a random N rows of data

In [26]:
from datasets import Dataset

def get_random_sample(dataset: Dataset, sample_size: int, seed: int = None) -> Dataset:
    """
    Returns a random sample of specified size from the given dataset.

    Args:
    dataset (Dataset): The dataset to sample from.
    sample_size (int): The size of the sample to return.
    seed (int, optional): The random seed for reproducibility. Defaults to None.

    Returns:
    Dataset: A new dataset containing the random sample.
    """
    shuffled_dataset = dataset.shuffle(seed=seed)
    sampled_dataset = shuffled_dataset.select(range(sample_size))
    # print(len(sampled_dataset))
    return sampled_dataset

# (END WIP)

Now after setting everything up we can start training.

We will read the dataset in chunks
1. for each chunk update the datasets in the trainer,
1. Load the latest version of the trained model
2. Train on that specific chunk
3. Save the snapshot

In [19]:
def get_latest_model_version():
  model_path = "/content/drive/MyDrive/Lab2/ColabData/Third"
  # load all subdirs
  subdir_list_2 = [os.path.join(model_path, d) for d in os.listdir(model_path) if os.path.isdir(os.path.join(model_path, d))]
  subdir_list_2 = [subdir for subdir in subdir_list_2 if "snapshot_step" in subdir]
  print(subdir_list_2)
  return get_latest_snapshot(subdir_list_2)


import re

def get_latest_snapshot(snapshot_list):
    # Extract step numbers and corresponding file paths
    step_files = {}
    for path in snapshot_list:
        # This regex matches any sequence of digits (\d+) following 'snapshot_step_'
        match = re.search(r'snapshot_step_(\d+)', path)
        if match:
            step = int(match.group(1))
            step_files[step] = path

    # Find the highest step number
    if step_files:
        latest_step = max(step_files.keys())
        return step_files[latest_step]
    else:
        return None


def load_snapshot(trainer):
    latest_snapshot = get_latest_model_version()
    # print(type(trainer))
    if latest_snapshot:
        print(f"Loading model from {latest_snapshot}")
        return latest_snapshot
    else:
        print("No snapshot found in the specified folder.")




In [ ]:
chunk_size2 = 1 # Use 1 because if not there can be problems with the offset
total_avail_chunks = len(subdir_list)
import torch
import time
start_time = time.time()
# To use one sample for all the eval's
# trainer.eval_dataset = get_random_sample(dataset_test, sample_size=200)
# test
remountGDrive()

for i, datasets_chunk in read_datasets_in_chunks(sub_dirs=subdir_list, chunk_size=chunk_size2, offset=27):
  # Update the datasets in trainer
  trainer.train_dataset = datasets_chunk
  # To randomize each time.
  trainer.eval_dataset = get_random_sample(dataset_test, sample_size=200)
  #print(len(trainer.eval_dataset))
  latest_snapshot = load_snapshot(trainer)
  step_number = re.search(r'(\d{1,3})$', latest_snapshot).group()
  step_number = int(step_number) + 1
  print("step: ", step_number)
  print("i: ", i)
  if int(i) != int(step_number):
    # for when we have already processed a chunk
    print("Chunk: " + str(i) + " has already been trained")
    torch.cuda.empty_cache()
    datasets_chunk.cleanup_cache_files()
    del datasets_chunk
    gc.collect()
    # the cache problematic with GDrive does only appear to happen on file upload to gdrive
    # remountGDrive()
    continue
  # Load the model only if I actually want to train
  trainer.model = WhisperForConditionalGeneration.from_pretrained(latest_snapshot)
  # just make sure these are still set (probably not necessary)
  trainer.model.config.forced_decoder_ids = None
  trainer.model.config.suppress_tokens = []

  # Train on the chunk
  trainer.train()


  print(f"processed dataset {step_number} of {total_avail_chunks}")

  # Save snapshot after processing each chunk
  save_snapshot(trainer, "/content/drive/MyDrive/Lab2/ColabData/Third", step_number)
  torch.cuda.empty_cache()
  # I set the train_dataset to None before the delete so there are no references to the original dataset remaining.
  trainer.train_dataset = []
  datasets_chunk.cleanup_cache_files()
  del datasets_chunk
  # we are remounting Google Drive because
  # remountGDrive()


  gc.collect()

  end_time = time.time()
  duration = end_time - start_time
  start_time = time.time()
  print(f"Iteration {i}: {duration} seconds")
  #break



Clearing cache in: /root/.config/Google/DriveFS/100531763657874180008/content_cache
Cache cleared.
Mounted at /content/drive
Loading datasets from 1 directories: ['/content/drive/MyDrive/Lab2/ColabData/Second/processed_chunk_28.dataset']
['/content/drive/MyDrive/Lab2/ColabData/Third/snapshot_step_1', '/content/drive/MyDrive/Lab2/ColabData/Third/snapshot_step_2', '/content/drive/MyDrive/Lab2/ColabData/Third/snapshot_step_3', '/content/drive/MyDrive/Lab2/ColabData/Third/snapshot_step_4', '/content/drive/MyDrive/Lab2/ColabData/Third/snapshot_step_5', '/content/drive/MyDrive/Lab2/ColabData/Third/snapshot_step_6', '/content/drive/MyDrive/Lab2/ColabData/Third/snapshot_step_7', '/content/drive/MyDrive/Lab2/ColabData/Third/snapshot_step_8', '/content/drive/MyDrive/Lab2/ColabData/Third/snapshot_step_9', '/content/drive/MyDrive/Lab2/ColabData/Third/snapshot_step_10', '/content/drive/MyDrive/Lab2/ColabData/Third/snapshot_step_11', '/content/drive/MyDrive/Lab2/ColabData/Third/snapshot_step_12', '/

Step,Training Loss,Validation Loss


And to finally top it off, let's run the evaluate function on the whole test_dataset.

In [ ]:
trainer.evaluate(dataset_test)